In [ ]:
import pandas as pd
import numpy as np
import datetime as datetime
import ta as ta
import warnings
warnings.filterwarnings("ignore")
import pdb
from decimal import Decimal
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.ticktype import TickTypeEnum
import threading
from ...lib import utils, constants
import datetime
import time
import signal
import sys
import pytz
import pyttsx3


#python -m navis.signals.mr.mr
#username: navispaper
#Flushit!234

port = 7496 ## 7496 is live, 7497 is paper trading
client_id = 404
#sp500_components = pd.read_csv("data/historical_intraday_data/sp500_components.csv")

retrade_timer = 3 * 60

trading_start_hour = 9
trading_start_minute = 30
trading_end_hour = 16
trading_end_minute = 0

size_diff_mult = 1
vol_adjustment_in_minutes = 0.1
tp_zScore_threshold = 0.1
option_volume_min_threshold_zScore = 0.5
window_size = 100
option_volume_jump_mult = 10

tp_aggregation_window = '5min'
ticker_list = constants.ticker_list_io

last_trigger_time_rumor = 0
silence_period = 2 * 60  # 5 minutes in seconds
alpha = 2/(window_size + 1)
target_timezone = pytz.timezone('US/Eastern')

log_file_location = 'logs/rumor_logs/%s_rumor_signal.csv' % (datetime.datetime.now(target_timezone).date())
with open(log_file_location, 'a') as file:
    file.write("RumorFuse,ticker, price_change_pct,self.tp_z_score,tp_zScore_threshold, vol_adjustment_in_minutes,atm_impliedVol,live_size['size_diff'],size_diff_mult,live_size['size_ema'],self.cv_zScore,option_volume_min_threshold_zScore")

class IBApi(EClient, EWrapper):

    def __init__(self, ib_client):
        EClient.__init__(self, self)
        self.ib_client = ib_client
        self.nextValidOrderId = 1
        self.subscription_req_id = 100
        self.live_price = {}
        self.live_size = {}
        self.ticker_given_reqId = {}
        self.reqId_given_ticker = {}
        self.atm_impliedVol = {}
        self.call_volume_df = pd.DataFrame([])
        self.put_volume_df = pd.DataFrame([])
        self.tp_z_score = {}

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextValidOrderId = orderId

    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        print(f"reqId: {reqId}, errorCode: {errorCode}, errorString: {errorString}, orderReject: {advancedOrderReject}")

    def tickGeneric(self, reqId, tickType, value):
        if tickType==24:
            ticker = self.ticker_given_reqId[reqId]
            print(ticker, value)

    def tickPrice(self, reqId, tickType, price, attrib):

        current_time = time.time()
        global last_trigger_time_macd
        global last_trigger_time_rsi
        global last_trigger_time_rsi_zScore

        self.reqIds(-1)
        ticker = self.ticker_given_reqId[reqId]
        if tickType==1:
            #print("time: %s, ticker: %s, price: %f " % (datetime.datetime.now(), ticker, price))
            self.live_price[ticker]['bid'] = float(price)
            try:
                self.live_price[ticker]['bid_ema'] = self.live_price[ticker]['bid']*alpha + self.live_price[ticker]['bid_ema']*(1-alpha)
            except:
                self.live_price[ticker]['bid_ema'] = self.live_price[ticker]['bid']
        if tickType==2:
            self.live_price[ticker]['ask'] = float(price)
            try:
                self.live_price[ticker]['ask_ema'] = self.live_price[ticker]['ask']*alpha + self.live_price[ticker]['ask_ema']*(1-alpha)
            except:
                self.live_price[ticker]['ask_ema'] = self.live_price[ticker]['ask']
        try:
            self.live_price[ticker]['mid'] = 0.5*(self.live_price[ticker]['bid']+self.live_price[ticker]['ask'])
        except:
            pass

    def tickSize(self, reqId, tickType, size):
        current_time = time.time()
        global last_trigger_time_rumor
        global last_trigger_time_tp

        self.reqIds(-1)
        ticker = self.ticker_given_reqId[reqId]
        now_ = datetime.datetime.now(target_timezone)
        current_time_hour_minute = datetime.datetime(now_.year, now_.month, now_.day, now_.hour, now_.minute, 0)

        if tickType == 8: ## live volume
            self.df_ul_volume.loc[current_time_hour_minute, ticker] = 100*int(size)
            try:
                tp = self.df_ul_volume[ticker].diff() * self.df_ul_prices[ticker].diff()
                tp_agg = tp.resample(tp_aggregation_window).sum()
                self.tp_z_score[ticker] = (tp_agg - tp_agg.median())/tp_agg.std()
                if current_time - last_trigger_time_macd >= silence_period:
                    if self.tp_z_score[ticker] > tp_zScore_threshold:
                        #self.text_to_speech("Big Buy trade pressure to buy in %s" % ticker)
                        last_trigger_time_tp = current_time
                if current_time - last_trigger_time_macd >= silence_period:
                    if self.tp_z_score[ticker] < -1*tp_zScore_threshold:
                        #self.text_to_speech("Big Sell trade pressure to sell in %s" % ticker)
                        last_trigger_time_tp = current_time
            except:
                pass
        if tickType == 21: ## 90-day avg volume
            self.live_size[ticker]['avg_share_volume'] = 100*int(size)
        if tickType == 29: ## call option volume
            self.call_volume_df.loc[current_time_hour_minute, ticker] = int(size)
            try:
                self.cv_zScore[ticker] = (self.call_volume_df[ticker].diff().iloc[-1] - self.call_volume_df[ticker].diff().mean()) / self.call_volume_df[ticker].diff().std()
                if self.cv_zScore[ticker] > option_volume_min_threshold_zScore:
                    if current_time - last_trigger_time_cv >= silence_period:
                        #self.text_to_speech("Large volume in call options in %s" % ticker)
                        last_trigger_time_cv = current_time
            except:
                pass
        if tickType == 30: ## put option volume
            self.put_volume_df.loc[current_time_hour_minute, ticker] = int(size)
            try:
                self.pv_zScore[ticker] = (self.put_volume_df[ticker].diff().iloc[-1] - self.put_volume_df[ticker].diff().mean()) / self.put_volume_df[ticker].diff().std()
                if self.pv_zScore[ticker] > option_volume_min_threshold_zScore:
                    if current_time - last_trigger_time_cv >= silence_period:
                        #self.text_to_speech("Large volume in put options in %s" % ticker)
                        last_trigger_time_cv = current_time
            except:
                pass
        if tickType == 87: ## average option volume
            self.live_size[ticker]['avg_option_volume'] = int(size)

        if tickType==0:
            self.live_size[ticker]['bid'] = int(size)

        if tickType==3:
                self.live_size[ticker]['ask'] = int(size)

        try:
            price_change_pct = self.live_price[ticker]['mid']/self.live_price[ticker]['bid_ema'] - 1
            if self.tp_z_score[ticker].iloc[-1] > tp_zScore_threshold and price_change_pct > vol_adjustment_in_minutes*self.atm_impliedVol[ticker]/15.7/6.5/60 and self.live_size[ticker]['size_diff'] > size_diff_mult*self.live_size[ticker]['size_ema'] and self.cv_zScore[ticker]>option_volume_min_threshold_zScore:
                if current_time - last_trigger_time_rumor >= silence_period:
                        self.text_to_speech("Rumor fuse: Big jump in stock price in %s" % ticker)
                        last_trigger_time_rumor = current_time
                        #now, reset ema's so it doesn't keep triggering
                        self.live_price[ticker]['bid_ema'] = self.live_price[ticker]['mid']
                        self.live_price[ticker]['ask_ema'] = self.live_price[ticker]['mid']

                        with open(log_file_location, 'a') as file:
                                file.write("RumorFuse,%s,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f" % (ticker, price_change_pct,self.tp_z_score[ticker].iloc[-1],tp_zScore_threshold, vol_adjustment_in_minutes, self.atm_impliedVol[ticker],self.live_size[ticker]['size_diff'],size_diff_mult,self.live_size[ticker]['size_ema'],self.cv_zScore[ticker],option_volume_min_threshold_zScore))

            if self.tp_z_score[ticker].iloc[-1] < -1*tp_zScore_threshold and price_change_pct < -1*vol_adjustment_in_minutes*self.atm_impliedVol[ticker]/15.7/6.5/60 and -1*self.live_size[ticker]['size_diff'] > size_diff_mult*self.live_size[ticker]['size_ema'] and self.pv_zScore[ticker]>option_volume_min_threshold_zScore:
                if current_time - last_trigger_time_rumor >= silence_period:
                    self.text_to_speech("Rumor fuse: Big drop in stock price in %s" % ticker)
                    last_trigger_time_rumor = current_time
                    #now, reset ema's so it doesn't keep triggering
                    self.live_price[ticker]['bid_ema'] = self.live_price[ticker]['mid']
                    self.live_price[ticker]['ask_ema'] = self.live_price[ticker]['mid']

                    with open(log_file_location, 'a') as file:
                        file.write("RumorFuse,%s,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f" % (ticker, price_change_pct,self.tp_z_score[ticker].iloc[-1],tp_zScore_threshold, vol_adjustment_in_minutes, self.atm_impliedVol[ticker],self.live_size[ticker]['size_diff'],size_diff_mult,self.live_size[ticker]['size_ema'],self.pv_zScore[ticker],option_volume_min_threshold_zScore))


        except Exception as err:
            #print(f"Unexpected {err=}, {type(err)=}")
            pass


    def delayed_action(self):
        #print("30 seconds have passed without blocking IBKR updates.")
        pass

    def tickGeneric(self, reqId, tickType, price):
        ticker = self.ticker_given_reqId[reqId]
        if tickType == 24:
            self.atm_impliedVol[ticker] = price


    def place_limit_order(self, order_id, symbol, direction, lmt_price, quantity):
        contract = utils.getStockContract(symbol)
        ord = Order()
        ord.orderId = order_id
        ord.action = direction
        ord.orderType = "LMT"
        ord.totalQuantity = quantity
        ord.lmtPrice = lmt_price
        ord.tif = "GTC"
        #ord.transmit = True
        self.placeOrder(ord.orderId, contract, ord)

    def place_mkt_order(self, order_id, symbol, direction, quantity):
        self.reqGlobalCancel()
        contract = utils.getStockContract(symbol)
        ord = Order()
        ord.orderId = order_id
        ord.action = direction
        ord.orderType = "MKT"
        ord.totalQuantity = quantity
        ord.tif = "GTC"
        #ord.transmit = True
        self.placeOrder(ord.orderId, contract, ord)

    def text_to_speech(self, text_to_speak):
        # --- Configuration ---
        #text_to_speak = "This is another text-to-speech example using pyttsx3, which works offline."
        # --- End Configuration ---

        try:
            # Initialize the TTS engine
            engine = pyttsx3.init()

            # --- Optional: Adjust properties (before saying text) ---

            # Set Rate (Speed)
            rate = engine.getProperty('rate')   # Get current speaking rate
            # print(f"Default Rate: {rate}")
            engine.setProperty('rate', 150)     # Setting up new voice rate (words per minute)

            # Set Volume
            volume = engine.getProperty('volume') # Get current volume level (min=0, max=1)
            # print(f"Default Volume: {volume}")
            engine.setProperty('volume', 1.0)    # Setting up volume level between 0 and 1

            # Set Voice (Optional - depends on installed voices)
            voices = engine.getProperty('voices') # Get list of available voices
            # print("Available Voices:")
            #for voice in voices:
             #   print(f" - ID: {voice.id}, Name: {voice.name}, Lang: {voice.languages}")
            # Uncomment the next line to try setting a specific voice by ID
            # engine.setProperty('voice', voices[1].id) # Example: Set to the second voice found

            # --- End Optional Properties ---


            print(f"%s: '{text_to_speak}'" % datetime.datetime.now(target_timezone))

            # Queue the text to be spoken
            engine.say(text_to_speak)

            # Process the queued commands and wait for speech to finish
            engine.runAndWait()

            print("Speech finished.")

            # If you want to save to a file instead of speaking directly:
            # output_filename_pyttsx3 = "output_pyttsx3.wav" # Can save as wav, mp3 etc.
            # print(f"Saving speech to {output_filename_pyttsx3}...")
            # engine.save_to_file(text_to_speak, output_filename_pyttsx3)
            # engine.runAndWait() # Need this to actually perform the save
            # print("Save complete.")

        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            # Optional: Properly shut down the engine if it was initialized
            if 'engine' in locals() and engine._inLoop:
                engine.endLoop()

class RUMOR:

    def __init__(self):

        self.ib = IBApi(self)
        self.ib.connect("127.0.0.1", port, client_id)  ## 7496 is live, 7497 is paper trading
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        ib_thread.start()
        time.sleep(5)

    def run_loop(self):
        self.ib.run()

    def d1(self, strike, stock_price, vol, tte_in_days):
        return ( np.log(stock_price/strike) + 0.5*vol*vol*tte_in_days/252) / (vol * np.sqrt(tte_in_days/252))

    def delta(self, strike, stock_price, vol, call_put, tte_in_days):
        d_1 = self.d1(strike, stock_price, vol, tte_in_days)
        if call_put == 'call':
            delta = norm.cdf(d_1)
        else:
            delta = -1*norm.cdf(-1*d_1)
        return delta

    def getStrikeGivenDelta(self, target_delta, stock_price, call_put, tte_in_days):
        target_strike = np.nan
        vol = 0.2
        for strike in range(round(stock_price) - 10, round(stock_price)+10):
            delta_ = self.delta(strike, stock_price, vol, call_put ,tte_in_days)
            if target_delta-0.06 <= delta_ <= target_delta+0.06:
                target_strike = strike
        return target_strike

    def get_tte_in_days(self, date, t1):
        t2 = datetime.time(17, 15)
        tte_in_seconds = (datetime.datetime.combine(date, t2) - datetime.datetime.combine(date, t1)).seconds
        return tte_in_seconds/60/60/24

    def algo(self):

        #ticker_list = sp500_components['Symbol'].values
        for ticker in ticker_list:#np.append(ticker_list[0:100],['SPY', 'QQQ', 'IWM']):
            print(ticker)
            self.ib.live_price[ticker] = {}
            self.ib.live_size[ticker] = {}
            self.ib.subscription_req_id += 1
            self.ib.ticker_given_reqId[self.ib.subscription_req_id] = ticker
            self.ib.reqId_given_ticker[ticker] = self.ib.subscription_req_id
            self.ib.reqMktData(self.ib.subscription_req_id, utils.getStockContract(ticker), "165,106,100", False, False, [])
            time.sleep(1)
            self.ib.reqIds(0)
        pdb.set_trace()
        while True:


            a=1

    def start(self):

        t1 = threading.Thread(target=self.algo, daemon=True)
        all_threads = [t1]
        for t in all_threads:
            t.start()
        for t in all_threads:
            t.join()

    def disconnect(self):
        self.ib.disconnect()

    def signal_handler(self, sig, _):
        print("CTRL C")
        self.disconnect()
        sys.exit(0)

if __name__ == '__main__':

    rumor = RUMOR()
    try:
        signal.signal(signal.SIGINT, rumor.signal_handler)
        rumor.start()
    except KeyboardInterrupt:
        pass
    finally:
        rumor.disconnect()

